# Adeptus Optimus Proto

# Calculus

In [2]:
import scipy.special
import numpy as np
import random
import re
import json
import math

In [3]:
f'%.{1}E' % 0.03345

'3.3E-02'

In [4]:
def float_eq(a, b, n_same_decimals=4):
    return f'%.{n_same_decimals}E' % a == f'%.{n_same_decimals}E' % b

assert(float_eq(1, 1.01, 1))
assert(float_eq(0.3333, 0.3334, 2))
assert(float_eq(0.03333, 0.03334, 2))
assert(not float_eq(0.3333, 0.334, 2))
assert(not float_eq(0.03333, 0.0334, 2))

In [5]:
# Utils
def map_7_to_None(v):
    return None if v == 7 else v

class RequirementFailError(Exception):
    pass

def require(predicate, error_message):
    if not(predicate):
        raise RequirementFailError(error_message)

require(True, "bla")

thrown_message = ""
try:
    require(False, "bla")
except RequirementFailError as e:
    thrown_message = str(e)
assert(thrown_message == "bla")
    
    
    
def roll_D6():
    return random.randint(1, 6)
assert(len({roll_D6() for _ in range(1000)}) == 6)

def roll_D3():
    return (random.randint(1, 6) + 1) // 2
assert(len({roll_D3() for _ in range(1000)}) == 3)

class DiceExpr:
    def __init__(self, n, dices_type=None):
        self.n = n
        self.dices_type = dices_type
        
        if self.dices_type is None:
            self.avg = n
        else:
            self.avg = n * (self.dices_type + 1)/2
    
    def roll(self):
        if self.dices_type is None:
            return self.n
        else:
            if self.dices_type == 3:
                return sum([roll_D3() for _ in range(self.n)])
            elif self.dices_type == 6:
                return sum([roll_D6() for _ in range(self.n)])
            else:
                raise AttributeError(f"Unsupported dices_type D{self.dices_type}")
    
        
    def __str__(self):
        if self.dices_type is None:
            return str(self.n)
        else:
            return f"{self.n if self.n > 1 else ''}D{self.dices_type}"

        
assert(str(DiceExpr(5, 3)) == "5D3")
assert(str(DiceExpr(1, 6)) == "D6")
assert(str(DiceExpr(10, None)) == "10")

dice_5D3 = DiceExpr(5, 3)
assert(len({dice_5D3.roll() for _ in range(10000)}) == 5*3 -5 + 1)
dice_4D6 = DiceExpr(4, 6)
assert(len({dice_4D6.roll() for _ in range(10000)}) == 4*6 -4 + 1)          
    
def parse_dice_expr(d, complexity_threshold=36):
    assert(type(d) is str)
    groups = re.fullmatch(r"([1-9][0-9]*)?D([36])?|([0-9]+)", d)
    res = None
    try:
        if groups.group(1) is None:
            n_dices = 1
        else:
            n_dices = int(groups.group(1))
        dices_type = int(groups.group(2))
        if(n_dices*(1 if dices_type is None else dices_type) > complexity_threshold):
            res = None
        else:
            res = DiceExpr(n_dices, dices_type)
    except TypeError:
        try:
            flat = int(groups.group(3))
            res = DiceExpr(flat)
        except TypeError:
            res = None
    finally:
        return res
    
assert(parse_dice_expr("4D3").avg == 8)
assert(parse_dice_expr("5").avg == 5)
assert(parse_dice_expr("D7") is None)
assert(parse_dice_expr("0D6") is None)
assert(parse_dice_expr("0").avg == 0)
assert(parse_dice_expr("7D6") is None)
assert(parse_dice_expr("D3").avg == 2)
assert(parse_dice_expr("3D3").avg == 6)
assert(parse_dice_expr("D6").avg == 3.5)

def parse_roll(roll):
    res = re.fullmatch(r"([23456])\+", roll)
    if res is None:
        return None
    else:
        return int(res.group(1))

assert(parse_roll("1+") is None)
assert(parse_roll("1+") is None)
assert(parse_roll("2+") == 2)
assert(parse_roll("3+") == 3)
assert(parse_roll("6+") == 6)
assert(parse_roll("7+") is None)
assert(parse_roll("3") is None)



def prob_by_roll_result(dice_expr):
    if dice_expr.dices_type is None:
        return {dice_expr.n: 1}
    else:
        roll_results_counts = {}
        def f(n, current_sum):
            if n == 0:
                roll_results_counts[current_sum] = roll_results_counts.get(current_sum, 0) + 1
            else:
                for i in range(1, dice_expr.dices_type + 1):
                    f(n - 1, current_sum + i)
        f(dice_expr.n, 0)
        n_cases = sum(roll_results_counts.values())
        for key in roll_results_counts.keys():
            roll_results_counts[key] /= n_cases
        return roll_results_counts
    
assert(prob_by_roll_result(parse_dice_expr("D3")) == {1: 1/3, 2: 1/3, 3: 1/3})
assert(prob_by_roll_result(parse_dice_expr("7")) == {7: 1})
assert(float_eq(1, sum(prob_by_roll_result(parse_dice_expr("2D6")).values())))
assert(prob_by_roll_result(parse_dice_expr("2D6")) == {2: 1/36, 3: 2/36, 4: 3/36, 5: 4/36, 6: 5/36, 7: 6/36, 8: 5/36, 9: 4/36, 10: 3/36, 11: 2/36, 12: 1/36})

In [6]:
# Core Classes
class Bonuses:
    def __init__(self, to_hit, to_wound, props = None):
        assert(to_hit in {-1, 0, 1})
        assert(to_wound in {-1, 0, 1})

        self.to_hit = to_hit
        self.to_wound = to_wound
        self.props = props if props is not None else {}
        
    @classmethod
    def empty(cls):
        return Bonuses(0, 0)
    
Bonuses.empty().to_hit = 0
        
class Weapon:
    def __init__(self, hit, a, s, ap, d, bonuses, points=1):
        self.hit      = parse_dice_expr(hit)
        assert(self.hit is not None)
        self.a        = parse_dice_expr(a)
        assert(self.a is not None)
        self.s        = parse_dice_expr(s)
        assert(self.s is not None)
        self.ap       = parse_dice_expr(ap)
        assert(self.ap is not None)
        self.d        = parse_dice_expr(d)
        assert(self.d is not None)

        self.bonuses  = bonuses   
        assert(type(points) is int and points > 0)
        self.points   = points
        
        
Weapon(hit="5", a="2", s = "4D3", ap = "1", d="D3", bonuses=Bonuses.empty())

        
class Target:
    def __init__(self, t, sv, invu=None, fnp=None, w=1):

        
        assert(invu is None or (type(invu) is int and invu > 0 and invu <= 6))
        self.invu   = invu
        
        assert(fnp is None or (type(fnp) is int and fnp > 0 and fnp <= 6))
        self.fnp   = fnp
        
        assert(type(t) is int and t > 0)
        self.t    = t
        
        assert(type(sv) is int and sv > 0 and sv <= 6)
        self.sv   = sv
        
        assert(type(w) is int and w > 0)
        self.w   = w


Target(8, 3, 5, 6)
Target(8, 3)

In [7]:
# Engine v1

def compute_successes_ratio(modified_necessary_roll, auto_success_on_6=True):
    necessary_roll = modified_necessary_roll
    if modified_necessary_roll <= 1:
        necessary_roll = 2  # roll of 1 always fails
    if modified_necessary_roll >= 7:
        if auto_success_on_6:
            necessary_roll = 6  # roll of 6 always succeeds
        else:
            return 0
    return (7 - necessary_roll)/6

def compute_n_hits_ratio(weapon):
    return compute_successes_ratio(weapon.hit.avg - weapon.bonuses.to_hit)


assert(compute_n_hits_ratio(Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 5/6)
assert(compute_n_hits_ratio(Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses.empty())) == 1/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses.empty())) == 2/3)
assert(compute_n_hits_ratio(Weapon(hit="3", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/2)
assert(compute_n_hits_ratio(Weapon(hit="2", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(+1, 0))) == 5/6)  # roll of 1 always fails
assert(compute_n_hits_ratio(Weapon(hit="6", a="1", s = "4", ap = "1",  d="2D3", bonuses=Bonuses(-1, 0))) == 1/6)  # roll of 6 always succeeds

def compute_necessary_wound_roll(f, e):
    if f >= 2*e:
        return 2
    elif f > e:
        return 3
    elif f == e:
        return 4
    elif f <= e/2:
        return 6
    else:
        assert(f < e)
        return 5

def compute_n_wounds_ratios(weapon, target):
    return [prob*compute_successes_ratio(compute_necessary_wound_roll(f, target.t) - weapon.bonuses.to_wound)
     for f, prob in prob_by_roll_result(weapon.s).items()]

def compute_n_wounds_ratio(weapon, target):
    return sum(compute_n_wounds_ratios(weapon, target))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
) == 1/2)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
) == 1/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, -1)),
    Target(t=8, sv=3)
) == 1/6)  # roll of 6 always succeeds

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=2, sv=3)
) == 5/6)

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, +1)),
    Target(t=2, sv=3)
) == 5/6)  # roll of 1 always fails

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=12, sv=3)
) == 1/6)

assert(float_eq(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=1, sv=3)
), 5/6))

assert(compute_n_wounds_ratio(
    Weapon(hit="2", a="1", s = "2D3", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=4, sv=3)
)< 1/2)



def n_figs_slained(weapon, target):
    hit_ratio = compute_n_hits_ratio(weapon)
    wound_ratio = compute_n_wounds_ratio(weapon, target)
    return weapon.a.avg * hit_ratio * wound_ratio

def total_n_figs_slained(weapons_list, target):
    print("tatata")

In [8]:
n_figs_slained(
    Weapon(hit="5", a="12", s = "4", ap = "1", d="2D3", bonuses=Bonuses(+1, 0)),
    Target(t=8, sv=3)
)

1.0

In [9]:
# Engine v2
def dispatch_density_key(previous_density_key, next_density_prob):
    assert(type(previous_density_key) is int)
    assert(previous_density_key >= 0)
    assert(0 < next_density_prob and next_density_prob <= 1)
    n = previous_density_key
    p= next_density_prob
    return {k: scipy.special.comb(n, k)*p**k*(1-p)**(n-k) for k in range(0, n + 1)}

assert(dispatch_density_key(3, 0.5) == {0: 0.125, 1: 0.375, 2: 0.375, 3: 0.125})
# new version:
# 3 A, c 4, f 4 endu 4
# [0, 0, 0, 1] attacks density
def get_attack_density(weapon):
    assert(isinstance(weapon, Weapon))
    return {a: prob for a, prob in prob_by_roll_result(weapon.a).items()}
# [1/8, 3/8, 3/8 ,1/8] hit density
def get_hits_density(weapon, attack_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(attack_density, dict))
    hits_density = {}
    for a, prob_a in attack_density.items(): 
        # {1: 0.3333333333333333, 2: 0.3333333333333333, 3: 0.3333333333333333}
        for hit_roll, prob_hit_roll in prob_by_roll_result(weapon.hit).items():
            # {5: 1}
            hits_ratio = compute_successes_ratio(hit_roll - weapon.bonuses.to_hit)
            # 0.5
            for hits, prob_hits in dispatch_density_key(a, hits_ratio).items():
                hits_density[hits] = hits_density.get(hits, 0) + prob_hits*prob_hit_roll*prob_a
    return hits_density

# [......]  woud density
def get_wounds_density(weapon, target, hits_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(hits_density, dict))
    wounds_density = {}
    for hits, prob_hits in hits_density.items(): 
        for s_roll, prob_s_roll in prob_by_roll_result(weapon.s).items():
            wounds_ratio = compute_successes_ratio(compute_necessary_wound_roll(s_roll, target.t) - weapon.bonuses.to_wound)
            for wounds, prob_wounds in dispatch_density_key(hits, wounds_ratio).items():
                wounds_density[wounds] = wounds_density.get(wounds, 0) + prob_wounds*prob_s_roll*prob_hits
    return wounds_density

# [......] unsaved wounds density
def get_unsaved_wounds_density(weapon, target, wounds_density):
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(wounds_density, dict))
    unsaved_wounds_density = {}
    for wounds, prob_wounds in wounds_density.items(): 
        for ap_roll, prob_ap_roll in prob_by_roll_result(weapon.ap).items():
            save_roll = target.sv + ap_roll
            if target.invu is not None:
                save_roll = min(save_roll, target.invu)
            unsaved_wounds_ratio = 1 - compute_successes_ratio(save_roll, auto_success_on_6=False)
            for unsaved_wounds, prob_unsaved_wounds in dispatch_density_key(wounds, unsaved_wounds_ratio).items():
                unsaved_wounds_density[unsaved_wounds] = unsaved_wounds_density.get(unsaved_wounds, 0) + prob_unsaved_wounds*prob_ap_roll*prob_wounds
    return unsaved_wounds_density

# last step numeric averaging: damage roll + fnp
def get_avg_figs_fraction_slained(weapon, target, unsaved_wounds_density, N):
    """
    El famoso montecarlo approach
    :param N: number of simulations for each unsaved wounds entry
    """
    assert(isinstance(weapon, Weapon))
    assert(isinstance(target, Target))
    assert(isinstance(unsaved_wounds_density, dict))
    figs_fraction_slained_total_weighted = 0
    adapted_N = (N // target.w) + 1
    for unsaved_wounds, prob_unsaved_wounds in unsaved_wounds_density.items():
        figs_fraction_slained_total_sum = 0
        for _ in range(adapted_N):
            figs_fraction_slained_total = 0
            for start_health in range(1, target.w + 1):
                n_figs_slained = 0
                remaining_health = start_health
                for wound in range(unsaved_wounds):
                    damages = weapon.d.roll()
                    if target.fnp is not None:
                        for damage in range(damages):
                            if roll_D6() >= target.fnp:
                                damages -= 1  # fnp success
                    remaining_health -= damages
                    if remaining_health <= 0:
                        n_figs_slained += 1
                        remaining_health = target.w
                # e.g. remaining = 1,slained 2, w=3, start = 2: frac = 2 - (1 - 2/3) + (1 - 1/3) 
                first_slain_fraction = start_health/target.w
                remaining_fraction = remaining_health/target.w
                figs_fraction_slained_total += \
                    n_figs_slained - (1 - first_slain_fraction) + (1 - remaining_fraction)

            figs_fraction_slained_total_sum += figs_fraction_slained_total

        figs_fraction_slained_total_weighted += \
            prob_unsaved_wounds * figs_fraction_slained_total_sum/(adapted_N*target.w) # target_wounds is here because of start_health loop

    return figs_fraction_slained_total_weighted

def score_weapon_on_target(w, t, N):
    """
    avg_figs_fraction_slained by point
    """
    a_d = get_attack_density(w)
    assert(float_eq(sum(a_d.values()), 1))
    h_d = get_hits_density(w, a_d)
    assert(float_eq(sum(h_d.values()), 1))
    w_d = get_wounds_density(w, t, h_d)
    assert(float_eq(sum(w_d.values()), 1))
    uw_d = get_unsaved_wounds_density(w, t, w_d)
    assert(float_eq(sum(uw_d.values()), 1))
    return get_avg_figs_fraction_slained(w, t, uw_d, N)/w.points

# Sv=1 : ignore PA -1
wea = Weapon(hit="4", a="4", s = "4", ap = "1", d="3", bonuses=Bonuses(0, 0), points=120)
wea2 = Weapon(hit="4", a="4", s = "4", ap = "0", d="3", bonuses=Bonuses(0, 0), points=120)
tar = Target(t=4, sv=1, invu=5, fnp=6, w=16)
assert(abs(score_weapon_on_target(wea, tar, 1000) / score_weapon_on_target(wea2, tar, 100) - 1) <= 0.25)

def scores_to_comparison_score(score_a, score_b):
    if score_a > score_b:
        return + (1 - score_b / score_a)
    else:
        return - (1 - score_a / score_b)

assert(scores_to_comparison_score(10000, 1) == 0.9999) 
assert(scores_to_comparison_score(1, 10000) == -0.9999) 
assert(scores_to_comparison_score(1, 1) == 0) 

def y_dims_to_str(l):
    return f"""{l[0]}/{l[1]}/{"-" if l[2] is None else f"{l[2]}+"}"""

def x_dims_to_str(l):
    return  f"""{"-" if l[0] is None else f"{l[0]}+"}/{"-" if l[1] is None else f"{l[1]}+"}"""
def compute_matrix(weapon_a, weapon_b, N):    
    ws_ts_fnps = []
    for w, ts in zip(
        [1,2,3,4,6,8,10,12,16], 
        [
            [2,3,4],
            [2,3,4,5],
            [2,3,4,5,6],
            [2,3,4,5,6],
            [3,4,5,6,8],
            [4,5,6,8],
            [5,6,8],
            [6,8],
            [6,8]
        ]
    ) :
        fnps = [7] if w > 6 else [7, 6, 5]
        for fnp in fnps:
            for t in ts:
                ws_ts_fnps.append((t, w, fnp))
                
    ws_ts_fnps.sort(key=lambda e: e[2] * 10000 - e[0]*100 - e[1])
    ws_ts_fnps = list(map(lambda l: list(map(map_7_to_None, l)), ws_ts_fnps))
    
    print(list(map(y_dims_to_str, ws_ts_fnps)), )
    
    svs = []
    for invu in [2, 3, 4, 5, 6, 7]:
        for sv in range(1, min(invu + 1, 6 + 1)):
            svs.append((sv, invu))
    svs.sort(key=lambda e: -e[0]*10 + -e[1])
    svs = list(map(lambda l: list(map(map_7_to_None, l)), svs))
    
    print(list(map(x_dims_to_str, svs)))
    
    
    return  \
        [
            [
                scores_to_comparison_score(
                    score_weapon_on_target(
                        weapon_a, 
                        Target(t, sv, invu=invu, fnp=fnp, w=w),
                        N), 
                    score_weapon_on_target(
                    weapon_b, 
                    Target(t, sv, invu=invu, fnp=fnp, w=w),
                    N)
                )
                for sv, invu in svs
            ]
            for w, t, fnp in ws_ts_fnps
        ]

In [10]:
# SAG vs Battlewagon under KFF and fnp 6+
sag = Weapon(hit="5", a="2D6", s = "2D6", ap = "5", d="D6", bonuses=Bonuses(0, 0), points=120)
bat = Target(t=8, sv=4, invu=5, fnp=6, w=16)

# SAG vs Battlewagon under KFF and fnp 6+
bolt = Weapon(hit="3", a="2", s = "4", ap = "1", d="1", bonuses=Bonuses(0, 0), points=20)

In [10]:
w=len([1,2,3,4,6,8,10,12,16])
t=len([2,3,4,5,6,8])
svs=len(range(1, 19 + 1))
fnp=len(range(1, 2 + 1))
w*t*svs*fnp  # 3200

2052

In [11]:
w_t_fnp = sum([
    1 * len([2,3,4])*2,      #w=1 # w * t possible *2 if fnp variant
    1 * len([2,3,4,5])*2,    #w=2
    1 * len([2,3,4,5,6])*2,  #w=3
    1 * len([2,3,4,5,6])*2,  #w=4
    1 * len([3,4,5,6,8])*2,#w=6
    1 * len([4,5,6,8]),#w=8
    1 * len([5,6,8]),#w=10
    1 * len([6,8]),#w=16
    1 * len([6,8]) #w=28
])
w_t_fnp

55

In [12]:
svs*w_t_fnp

1045

In [13]:
w = sag
t = bat

In [14]:
a_d = get_attack_density(w)
a_d

{2: 0.027777777777777776,
 3: 0.05555555555555555,
 4: 0.08333333333333333,
 5: 0.1111111111111111,
 6: 0.1388888888888889,
 7: 0.16666666666666666,
 8: 0.1388888888888889,
 9: 0.1111111111111111,
 10: 0.08333333333333333,
 11: 0.05555555555555555,
 12: 0.027777777777777776}

In [15]:
h_d = get_hits_density(w, a_d)
h_d, sum(h_d.values())

({0: 0.09245826180349491,
  1: 0.2239853530307222,
  2: 0.26620813348361666,
  3: 0.21083128491947162,
  4: 0.12367835752228379,
  5: 0.05596105682474631,
  6: 0.019891044662844356,
  7: 0.005563176345069349,
  8: 0.0012047433299274988,
  9: 0.00019485804737601263,
  10: 2.210969797211731e-05,
  11: 1.5680636859657664e-06,
  12: 5.226878953219221e-08},
 1.0000000000000002)

In [16]:
w_d = get_wounds_density(w, t, h_d)
w_d, sum(w_d.values())

({0: 0.4029500867053239,
  1: 0.3367914778591636,
  2: 0.1689350056321175,
  3: 0.06478956365764589,
  4: 0.020154421512724424,
  5: 0.00512477251439623,
  6: 0.0010565739623414905,
  7: 0.00017367555510426272,
  8: 2.216603219186902e-05,
  9: 2.1100352269821476e-06,
  10: 1.405806713069297e-07,
  11: 5.83937585822815e-09,
  12: 1.1371713614420627e-10},
 1.0000000000000004)

In [17]:
uw_d = get_unsaved_wounds_density(w, t, w_d)
uw_d, sum(uw_d.values())

({0: 0.5366555241737849,
  1: 0.3162275735568333,
  2: 0.11078687649773596,
  3: 0.029102035242819145,
  4: 0.006064610751263366,
  5: 0.0010130132279732571,
  6: 0.00013499857838335794,
  7: 1.415632468850579e-05,
  8: 1.1407205971613996e-06,
  9: 6.802863136706233e-08,
  10: 2.8236457380877304e-09,
  11: 7.27678850725364e-11,
  12: 8.764573859500293e-13},
 1.0000000000000007)

# New last step

In [18]:
# last step numeric averaging: damage roll + fnp
def get_avg_figs_fraction_slained_per_unsaved_wound(weapon, target, N):
    """
    :param N: number of consecutive wounds resolved: 
              - N=1000 leads to a result precise at +- 1.5% 
              - N=10000 leads to a result precise at +- 0.5% 
    """
    assert (isinstance(weapon, Weapon))
    assert (isinstance(target, Target))
    n_figs_slained = 0
    remaining_health = target.w
    for _ in range(N):
        damages = weapon.d.roll()
        if target.fnp is not None:
            for damage in range(damages):
                if roll_D6() >= target.fnp:
                    damages -= 1  # fnp success
        remaining_health -= damages
        if remaining_health <= 0:
            n_figs_slained += 1
            remaining_health = target.w
    # e.g. remaining = 1,slained 2, w=3, frac = 2 + (1 - 1/3) 
    remaining_fraction = remaining_health / target.w
    return (n_figs_slained + (1 - remaining_fraction))/N

run = [get_avg_figs_fraction_slained_per_unsaved_wound(sag, bat, 5000) for _ in range(100)]
sum(run)/len(run), np.std(np.array(run)), np.std(run)/(sum(run)/len(run))

(0.16756237500000004, 0.001367121171248181, 0.00815887917110378)

In [19]:
%timeit get_presimu_avg_figs_fraction_slained_by_unsaved_wound(bolt, bat, 5000)

NameError: name 'get_presimu_avg_figs_fraction_slained_by_unsaved_wound' is not defined

In [ ]:
get_presimu_avg_figs_fraction_slained_by_unsaved_wound(bolt, bat, 2000)

In [20]:
def get_avg_of_density(d):
    l = [float(v) * float(p) for v, p in d.items()]
    return sum(l)

In [21]:
def get_avg_figs_fraction_slained_per_unsaved_wound(target_w, target_fnp, weapon_d, N):
    """
    :param N: number of consecutive wounds resolved:
          - N=1000 leads to a result precise at +- 1.5%
          - N=10000 leads to a result precise at +- 0.5%
    """
    # alternative:
    if weapon.d.dices_type is None:
        if target.fnp is None:
            return 1/math.ceil(w/d)
        else:
            fnp_fail_ratio = 1 - compute_successes_ratio(target.fnp)
            for pained_damage, prob_pained_damage in dispatch_density_key(weapon.d.n, fnp_fail_ratio):
                
                
    n_figs_slained = 0
    remaining_health = target_w
    for _ in range(N):
        damages = weapon_d.roll()
        if target_fnp is not None:
            for damage in range(damages):
                if roll_D6() >= target_fnp:
                    damages -= 1  # fnp success
        remaining_health -= damages
        if remaining_health <= 0:
            n_figs_slained += 1
            remaining_health = target_w
    # e.g. remaining = 1,slained 2, w=3, frac = 2 + (1 - 1/3)
    remaining_fraction = remaining_health / target_w
    return (n_figs_slained + (1 - remaining_fraction)) / N



IndentationError: expected an indented block (<ipython-input-21-207cda8fd6ee>, line 16)

In [ ]:
%time get_avg_figs_fraction_slained_per_unsaved_wound(target_w=3, target_fnp=5, weapon_d=DiceExpr(2, None), \
                                                      N=10000)

In [ ]:
get_avg_figs_fraction_slained_per_unsaved_wound(target_w=3, target_fnp=5, weapon_d=DiceExpr(1, 3), N=1000000)

In [ ]:
d = 3
w = 5
def exact(d, w):
    return 1/math.ceil(w/d)
assert(f(d=3, w=5) == 0.5)
assert(f(d=2, w=2) == 1)
assert(f(d=6, w=16) == 1/3)

In [ ]:
f(d=3, w=5)

In [ ]:
dispatch_density_key(2, 5/6)

In [ ]:
5*5/6/6

In [11]:
def float_eq(a, b, n_same_decimals=4):
    print(f'%.{n_same_decimals}E' % a, f'%.{n_same_decimals}E' % b)
    return f'%.{n_same_decimals}E' % a == f'%.{n_same_decimals}E' % b


In [49]:
1/math.ceil(4/2), 1/math.ceil(3/2)

(0.5, 0.5)

In [68]:
esperance_n_figs_slained_by_an_unsaved_wound(2, DiceExpr(1, 3), 3/6)

0.5

In [37]:
def update_slained_figs_ratios(n_unsaved_wounds_left,
                               current_wound_n_damages_left,
                               n_figs_slained_so_far,
                               remaining_target_wounds,
                               prob_node,
                               start_target_wounds,
                               fnp_fail_ratio,
                               n_figs_slained_weighted_ratios,
                               weapon_d,
                               target_fnp,
                               target_wounds,
                               n_unsaved_wounds_init,
                               prob_min_until_cut,
                               current_wound_init_n_damages):
    assert (remaining_target_wounds >= 0)
    assert (n_unsaved_wounds_left >= 0)
    assert (current_wound_n_damages_left >= 0)
    if prob_node == 0:
        return

    # resolve a model kill
    if remaining_target_wounds == 0:
        remaining_target_wounds = target_wounds
        n_figs_slained_so_far += 1
        # additionnal damages are not propagated to other models
        current_wound_n_damages_left = 0
        update_slained_figs_ratios(n_unsaved_wounds_left,
                                   current_wound_n_damages_left,
                                   n_figs_slained_so_far,
                                   remaining_target_wounds,
                                   prob_node,
                                   start_target_wounds,
                                   fnp_fail_ratio,
                                   n_figs_slained_weighted_ratios,
                                   weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
                                   n_unsaved_wounds_init=n_unsaved_wounds_init,
                                   prob_min_until_cut=prob_min_until_cut,
                                   current_wound_init_n_damages=current_wound_init_n_damages)
        return

    # leaf: no more damages to fnp no more wounds to consume or p(leaf) < threshold
    if prob_node < prob_min_until_cut or (n_unsaved_wounds_left == 0 and current_wound_n_damages_left == 0):
        # print("leaf: n_figs_slained_so_far =", n_figs_slained_so_far, "remaining_target_wounds=", remaining_target_wounds )
        if current_wound_n_damages_left > 0:
            # wounds not used when branch is cut
            unused_unsaved_wounds_portion = n_unsaved_wounds_left + current_wound_n_damages_left/current_wound_init_n_damages
        else:
            unused_unsaved_wounds_portion = n_unsaved_wounds_left
        if n_unsaved_wounds_init == unused_unsaved_wounds_portion:
            assert(n_figs_slained_so_far == 0)
        else:
            used_unsaved_wounds_portion = n_unsaved_wounds_init - unused_unsaved_wounds_portion
            assert(used_unsaved_wounds_portion > 0)
            n_figs_slained_weighted_ratios.append(
                # prob, n_figs_slained_ratio_per_wound
                (
                        prob_node ,
                        (n_figs_slained_so_far +
                         (-1 + start_target_wounds / target_wounds) +  # portion of the first model cleaned
                         (1 - remaining_target_wounds / target_wounds)) /  # portion of the last model injured
                        (used_unsaved_wounds_portion)
                )
            )
        return

    # consume a wound
    if current_wound_n_damages_left == 0:
        n_unsaved_wounds_left -= 1
        # random doms handling
        for d, prob_d in prob_by_roll_result(weapon_d).items():
            current_wound_n_damages_left = d
            update_slained_figs_ratios(n_unsaved_wounds_left,
                                       current_wound_n_damages_left,
                                       n_figs_slained_so_far,
                                       remaining_target_wounds,
                                       prob_node * prob_d,
                                       start_target_wounds,
                                       fnp_fail_ratio,
                                       n_figs_slained_weighted_ratios,
                                       weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
                                       n_unsaved_wounds_init=n_unsaved_wounds_init,
                                       prob_min_until_cut=prob_min_until_cut,
                                       current_wound_init_n_damages=current_wound_n_damages_left)
        return

    # FNP success
    update_slained_figs_ratios(
        n_unsaved_wounds_left,
        current_wound_n_damages_left - 1,
        n_figs_slained_so_far,
        remaining_target_wounds,
        prob_node * (1 - fnp_fail_ratio),
        start_target_wounds,
        fnp_fail_ratio,
        n_figs_slained_weighted_ratios,
        weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
        n_unsaved_wounds_init=n_unsaved_wounds_init,
        prob_min_until_cut=prob_min_until_cut,
        current_wound_init_n_damages=current_wound_init_n_damages)

    # FNP fail
    update_slained_figs_ratios(
        n_unsaved_wounds_left,
        current_wound_n_damages_left - 1,
        n_figs_slained_so_far,
        remaining_target_wounds - 1,
        prob_node * fnp_fail_ratio,
        start_target_wounds,
        fnp_fail_ratio,
        n_figs_slained_weighted_ratios,
        weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
        n_unsaved_wounds_init=n_unsaved_wounds_init,
        prob_min_until_cut=prob_min_until_cut,
        current_wound_init_n_damages=current_wound_init_n_damages)


def compute_slained_figs_ratios(weapon_d, target_fnp, target_wounds,
                                n_unsaved_wounds_init=32,
                                prob_min_until_cut=0.0001):
    n_figs_slained_weighted_ratios = []
    fnp_fail_ratio = 1 if target_fnp is None else 1 - compute_successes_ratio(target_fnp)
    for start_target_wounds in range(target_wounds, target_wounds + 1):
        update_slained_figs_ratios(
            n_unsaved_wounds_left=n_unsaved_wounds_init,
            current_wound_n_damages_left=0,
            n_figs_slained_so_far=0,
            remaining_target_wounds=start_target_wounds,
            prob_node=1,
            start_target_wounds=start_target_wounds,
            fnp_fail_ratio=fnp_fail_ratio,
            n_figs_slained_weighted_ratios=n_figs_slained_weighted_ratios,
            weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
            n_unsaved_wounds_init=n_unsaved_wounds_init,
            prob_min_until_cut=prob_min_until_cut,
            current_wound_init_n_damages=0)
    # print(n_figs_slained_weighted_ratios)
    print(f"{len(n_figs_slained_weighted_ratios)/1} leafs by single tree, for depth={n_unsaved_wounds_init}")
    return sum(map(lambda tup: tup[0] * tup[1], n_figs_slained_weighted_ratios))/1

# FNP
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1), 6, 1), 5/6, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1), 5, 1), 4/6, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1), 4, 1), 0.5, 0))
# on W=2
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1), None, 2), 0.5, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(2), None, 2), 1, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(2, 3), None, 2), 1, 0))
# random doms
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1, 6), None, 35), 0.1, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1, 6), 4, 70, n_unsaved_wounds_init=32, prob_min_until_cut=0.0001), 0.025, 0))
assert(float_eq(compute_slained_figs_ratios(DiceExpr(1, 6), 5, 70, n_unsaved_wounds_init=32, prob_min_until_cut=0.0001), 2/60, 0))

22514.0 leafs by single tree, for depth=32
8E-01 8E-01
15719.0 leafs by single tree, for depth=32
7E-01 7E-01
16384.0 leafs by single tree, for depth=32
5E-01 5E-01
1.0 leafs by single tree, for depth=32
5E-01 5E-01
1.0 leafs by single tree, for depth=32
1E+00 1E+00
26933.0 leafs by single tree, for depth=32
1E+00 1E+00
46656.0 leafs by single tree, for depth=32
1E-01 1E-01
24016.0 leafs by single tree, for depth=32
3E-02 3E-02
22282.0 leafs by single tree, for depth=32
3E-02 3E-02


In [24]:
2/30

0.06666666666666667

In [ ]:
DiceExpr

In [93]:
%time compute_slained_figs_ratios(DiceExpr(1, 6), 5/6, 2, n_unsaved_wounds_init=3, prob_min_until_cut=0)

158213.0 leafs by single tree, for depth=3
CPU times: user 182 ms, sys: 0 ns, total: 182 ms
Wall time: 181 ms


0.2665038738836544

In [45]:
(0.2222*1+0.222+0.444*2)/2

0.6661

In [71]:
%time compute_slained_figs_ratios(DiceExpr(1, 3), None, 3, n_unsaved_wounds_init=6, prob_min_until_cut=0)

729.0 leafs by single tree, for depth=6
CPU times: user 2.74 ms, sys: 0 ns, total: 2.74 ms
Wall time: 2.66 ms


0.5746075293400396

In [64]:
%time compute_slained_figs_ratios(DiceExpr(1, 3), None, 3, n_unsaved_wounds_init=6, prob_min_until_cut=0.01)

243.0 leafs by single tree, for depth=6
CPU times: user 811 µs, sys: 0 ns, total: 811 µs
Wall time: 768 µs


0.28566529492455395

In [40]:
%time compute_slained_figs_ratios(DiceExpr(1, 3), 2, 3, n_unsaved_wounds_init=6, prob_min_until_cut=0)

3492074.0 leafs by single tree, for depth=6
CPU times: user 4.07 s, sys: 128 ms, total: 4.19 s
Wall time: 4.19 s


0.10920222031093094

In [41]:
%time compute_slained_figs_ratios(DiceExpr(1, 3), 2, 3, n_unsaved_wounds_init=6, prob_min_until_cut=0.0001)

20138.0 leafs by single tree, for depth=6
CPU times: user 30.4 ms, sys: 0 ns, total: 30.4 ms
Wall time: 30 ms


0.12574835986915847

In [32]:
# FNP
(
    compute_slained_figs_ratios(DiceExpr(2), 2, 5, n_unsaved_wounds_init=10, prob_min_until_cut=0.000),
    compute_slained_figs_ratios(DiceExpr(2), 2, 5, n_unsaved_wounds_init=10, prob_min_until_cut=0.00001),
)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [22]:
%time compute_slained_figs_ratios(DiceExpr(2), 2, 5, n_unsaved_wounds_init=100, prob_min_until_cut=0.0001)

23394.0 leafs by single tree, for depth=100
CPU times: user 48.7 ms, sys: 4.09 ms, total: 52.8 ms
Wall time: 51.4 ms


0.07567009146605616

In [130]:
%time assert (float_eq(compute_slained_figs_ratios(DiceExpr(1), 6, 1, n_unsaved_wounds_init=16, prob_min_until_cut=0), 5 / 6, 4))


65536.0 leafs by single tree, for depth=16
8.3333E-01 8.3333E-01
CPU times: user 118 ms, sys: 0 ns, total: 118 ms
Wall time: 117 ms


In [112]:
%time  (float_eq(compute_slained_figs_ratios(DiceExpr(1), 2, 1, n_unsaved_wounds_init=10, prob_min_until_cut=0.00003), 1 / 6, 1))

8.0 leafs by single tree, for depth=3
1.7E-01 1.7E-01
CPU times: user 1.13 ms, sys: 0 ns, total: 1.13 ms
Wall time: 772 µs


True

In [70]:
%time float_eq(compute_slained_figs_ratios(DiceExpr(2, 6), None, 6, \
                                           n_unsaved_wounds_init=100, \
                                           prob_min_until_cut=0.00001), 1, 10)

423251.0 leafs by single tree, for depth=100
8.0251331257E-01 1.0000000000E+00
CPU times: user 814 ms, sys: 6.41 ms, total: 820 ms
Wall time: 819 ms


False

In [72]:
assert (float_eq(compute_slained_figs_ratios(DiceExpr(1), 5, 1), 4 / 6, 4))

8.0 leafs by single tree, for depth=3


In [22]:
assert (float_eq(compute_slained_figs_ratios(DiceExpr(1), None, 1), 1, 3))
assert (float_eq(compute_slained_figs_ratios(DiceExpr(2), None, 1), 1, 3))

1.0 leafs by single tree, for depth=10
1.0 leafs by single tree, for depth=10


In [23]:
assert (float_eq(compute_slained_figs_ratios(DiceExpr(2), None, 5), 1 / 3, 0))

1.0 leafs by single tree, for depth=10


In [24]:
assert (float_eq(compute_slained_figs_ratios(DiceExpr(3), None, 5), 0.5, 1))
assert (float_eq(compute_slained_figs_ratios(DiceExpr(3), None, 5), 0.5, 1))

1.0 leafs by single tree, for depth=10
1.0 leafs by single tree, for depth=10


In [25]:
assert(compute_slained_figs_ratios(DiceExpr(2, 6), None, 7, \
                                           n_unsaved_wounds_init=100, \
                                           prob_min_until_cut=0.01) < 0.7)

491.0 leafs by single tree, for depth=100


In [26]:
assert(float_eq(compute_slained_figs_ratios(DiceExpr(2, 6), None, 70, \
                                           n_unsaved_wounds_init=100, \
                                           prob_min_until_cut=0.01), 0.1, 0))

491.0 leafs by single tree, for depth=100


In [119]:
%time float_eq(compute_slained_figs_ratios(DiceExpr(2, 6), 4, 70, \
                                           n_unsaved_wounds_init=100, \
                                           prob_min_until_cut=0.01), 0.1/2, 3)

152.0 leafs by single tree, for depth=100
5.000E-02 5.000E-02
CPU times: user 544 µs, sys: 0 ns, total: 544 µs
Wall time: 409 µs


True

In [115]:
# 1, 3
(compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=100),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=19),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=18),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=17),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=16),
 compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=15),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=14),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=13),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=12),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=11),
compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=10),
 compute_slained_figs_ratios(DiceExpr(2), None, 5, n_unsaved_wounds_init=9))

1.0 leafs by single tree, for depth=100
1.0 leafs by single tree, for depth=19
1.0 leafs by single tree, for depth=18
1.0 leafs by single tree, for depth=17
1.0 leafs by single tree, for depth=16
1.0 leafs by single tree, for depth=15
1.0 leafs by single tree, for depth=14
1.0 leafs by single tree, for depth=13
1.0 leafs by single tree, for depth=12
1.0 leafs by single tree, for depth=11
1.0 leafs by single tree, for depth=10
1.0 leafs by single tree, for depth=9


(0.33399999999999996,
 0.3368421052631579,
 0.3377777777777778,
 0.3364705882352942,
 0.3375,
 0.3386666666666666,
 0.3371428571428572,
 0.3384615384615385,
 0.33999999999999997,
 0.3381818181818182,
 0.33999999999999997,
 0.3422222222222222)

In [ ]:
def smooth_compute_slained_figs_ratios(weapon_d, target_fnp, target_wounds, n_unsaved_wounds_init_max=7, n_variants=3):
        return sum(
            [
                compute_slained_figs_ratios(
                    weapon_d,
                    target_fnp, 
                    target_wounds, 
                    n_unsaved_wounds_init_max - i)
                for i in range(n_variants)
            ])/n_variants
    
assert(float_eq(smooth_compute_slained_figs_ratios(
    DiceExpr(3), 
    None, 
    5,
    n_unsaved_wounds_init_max=17,
    n_variants=3), 0.5, 2))

In [15]:
%time compute_slained_figs_ratios(DiceExpr(2, 6), None, 1, n_unsaved_wounds_init=4)

CPU times: user 53.5 ms, sys: 0 ns, total: 53.5 ms
Wall time: 51.9 ms


0.9999999999999936

In [ ]:
%time compute_slained_figs_ratios(DiceExpr(2, 6), None, 1)

In [26]:
prob_by_roll_result(DiceExpr(1))

{1: 1}

In [34]:
float_eq(1.4, 1.6, 0)

1E+00 2E+00


False

# Attempt to elagate

In [ ]:
def esperance_n_figs_slained_by_an_unsaved_wound(target_wounds, weapon_d, fnp_fail_ratio):
    n_figs_slained_by_a_full_unsaved_wound_at_avg_damage = 1/math.ceil(target_wounds / weapon_d.avg)
    return  n_figs_slained_by_a_full_unsaved_wound_at_avg_damage* fnp_fail_ratio

def update_slained_figs_ratios(n_unsaved_wounds_left,
                               current_wound_n_damages_left,
                               n_figs_slained_so_far,
                               remaining_target_wounds,
                               prob_node,
                               start_target_wounds,
                               fnp_fail_ratio,
                               n_figs_slained_weighted_ratios,
                               weapon_d,
                               target_fnp,
                               target_wounds,
                               n_unsaved_wounds_init,
                               prob_min_until_cut,
                               current_wound_init_n_damages):
    assert (remaining_target_wounds >= 0)
    assert (n_unsaved_wounds_left >= 0)
    assert (current_wound_n_damages_left >= 0)
    if prob_node == 0:
        return

    # resolve a model kill
    if remaining_target_wounds == 0:
        remaining_target_wounds = target_wounds
        n_figs_slained_so_far += 1
        # additionnal damages are not propagated to other models
        current_wound_n_damages_left = 0
        update_slained_figs_ratios(n_unsaved_wounds_left,
                                   current_wound_n_damages_left,
                                   n_figs_slained_so_far,
                                   remaining_target_wounds,
                                   prob_node,
                                   start_target_wounds,
                                   fnp_fail_ratio,
                                   n_figs_slained_weighted_ratios,
                                   weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
                                   n_unsaved_wounds_init=n_unsaved_wounds_init,
                                   prob_min_until_cut=prob_min_until_cut,
                                   current_wound_init_n_damages=current_wound_init_n_damages)
        return

    # leaf: no more damages to fnp no more wounds to consume or p(leaf) < threshold
    if prob_node < prob_min_until_cut or (n_unsaved_wounds_left == 0 and current_wound_n_damages_left == 0):
        # print("leaf: n_figs_slained_so_far =", n_figs_slained_so_far, "remaining_target_wounds=", remaining_target_wounds )
        if current_wound_n_damages_left > 0:
            # wounds not used when branch is cut
            unused_unsaved_wounds_portion = n_unsaved_wounds_left + current_wound_n_damages_left/current_wound_init_n_damages
        else:
            unused_unsaved_wounds_portion = n_unsaved_wounds_left
        if n_unsaved_wounds_init == unused_unsaved_wounds_portion:
            assert(n_figs_slained_so_far == 0)
        else:
            used_unsaved_wounds_portion = n_unsaved_wounds_init - unused_unsaved_wounds_portion
            assert(used_unsaved_wounds_portion > 0)
            n_figs_slained_weighted_ratios.append(
                # prob, n_figs_slained_ratio_per_wound
                (
                        prob_node ,
                        ((n_figs_slained_so_far +
                         (-1 + start_target_wounds / target_wounds) +  # portion of the first model cleaned
                         (1 - remaining_target_wounds / target_wounds)) *  # portion of the last model injured
                        (used_unsaved_wounds_portion)
                        + esperance_n_figs_slained_by_an_unsaved_wound(
                            target_wounds,
                            weapon_d, 
                            fnp_fail_ratio)
                         *unused_unsaved_wounds_portion)/
                    n_unsaved_wounds_init**2
                )
            )
        return

    # consume a wound
    if current_wound_n_damages_left == 0:
        n_unsaved_wounds_left -= 1
        # random doms handling
        for d, prob_d in prob_by_roll_result(weapon_d).items():
            current_wound_n_damages_left = d
            update_slained_figs_ratios(n_unsaved_wounds_left,
                                       current_wound_n_damages_left,
                                       n_figs_slained_so_far,
                                       remaining_target_wounds,
                                       prob_node * prob_d,
                                       start_target_wounds,
                                       fnp_fail_ratio,
                                       n_figs_slained_weighted_ratios,
                                       weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
                                       n_unsaved_wounds_init=n_unsaved_wounds_init,
                                       prob_min_until_cut=prob_min_until_cut,
                                       current_wound_init_n_damages=current_wound_n_damages_left)
        return

    # FNP success
    update_slained_figs_ratios(
        n_unsaved_wounds_left,
        current_wound_n_damages_left - 1,
        n_figs_slained_so_far,
        remaining_target_wounds,
        prob_node * (1 - fnp_fail_ratio),
        start_target_wounds,
        fnp_fail_ratio,
        n_figs_slained_weighted_ratios,
        weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
        n_unsaved_wounds_init=n_unsaved_wounds_init,
        prob_min_until_cut=prob_min_until_cut,
        current_wound_init_n_damages=current_wound_init_n_damages)

    # FNP fail
    update_slained_figs_ratios(
        n_unsaved_wounds_left,
        current_wound_n_damages_left - 1,
        n_figs_slained_so_far,
        remaining_target_wounds - 1,
        prob_node * fnp_fail_ratio,
        start_target_wounds,
        fnp_fail_ratio,
        n_figs_slained_weighted_ratios,
        weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
        n_unsaved_wounds_init=n_unsaved_wounds_init,
        prob_min_until_cut=prob_min_until_cut,
        current_wound_init_n_damages=current_wound_init_n_damages)


def compute_slained_figs_ratios(weapon_d, target_fnp, target_wounds, n_unsaved_wounds_init=3,
                                prob_min_until_cut=0):
    n_figs_slained_weighted_ratios = []
    fnp_fail_ratio = 1 if target_fnp is None else 1 - compute_successes_ratio(target_fnp)
    for start_target_wounds in range(target_wounds, target_wounds + 1):
        update_slained_figs_ratios(
            n_unsaved_wounds_left=n_unsaved_wounds_init,
            current_wound_n_damages_left=0,
            n_figs_slained_so_far=0,
            remaining_target_wounds=start_target_wounds,
            prob_node=1,
            start_target_wounds=start_target_wounds,
            fnp_fail_ratio=fnp_fail_ratio,
            n_figs_slained_weighted_ratios=n_figs_slained_weighted_ratios,
            weapon_d=weapon_d, target_fnp=target_fnp, target_wounds=target_wounds,
            n_unsaved_wounds_init=n_unsaved_wounds_init,
            prob_min_until_cut=prob_min_until_cut,
            current_wound_init_n_damages=0)
    # print(n_figs_slained_weighted_ratios)
    print(f"{len(n_figs_slained_weighted_ratios)/1} leafs by single tree, for depth={n_unsaved_wounds_init}")
    return sum(map(lambda tup: tup[0] * tup[1], n_figs_slained_weighted_ratios))/1